In [1]:
import numpy as np

from weight import *
from utils import save, load
from library import *
from process_library_terms import *

In [2]:
%%prun # profiling

### load data
# vicsek:
#arrays.npy - normal. arrays2.npy - very few particles. arrays3.npy - high density. arrays4.npy - high density, low noise.
#arrays5.npy - small domain, one clump
#datasets = ['arrays.npy', 'arrays2.npy', 'arrays3.npy']#, 'arrays5.npy']
#datasets = ['data_bu.npy'] # burgers' equation data
## it may be interesting to reintroduce regression on multiple datasets,
## but for now we're just not going to do it

#dataset = 'data_bu.npy'
#dataset = 'data_cbu.npy'
#dataset = 'lamp2.npy'
#dataset = 'lamp2_1e-4.npy'
dataset = 'lamp_16x.npy'
#import scipy.io
#dataset = 'gas_1000_4_0.mat'
#dataset = 'gas_1000_16_4.mat'
#dataset = 'gas_5000_24_4.mat'
#matdata = scipy.io.loadmat(dataset)
#positions = matdata['pos_data']
#vs = matdata['vs_data']
#deltat = matdata['dt'][0][0]
#dims = matdata['dims'][0]
#for i, dset in enumerate(datasets):
positions, vs, deltat, dims = load(dataset, 4)
#positions = positions[0:2, :, :]
#vs = vs[0:2, :, :]
deltat = float(deltat)
world_size = np.array([dims[0], dims[0], dims[1]])
print(world_size)
cg_res = 200#200 - lamp; 20 - Jason; 5 - burgers
#world_size[2] = world_size[2]-200 # start at frame 200
#print(positions.shape)
#positions = positions[:, :, 200:]
#vs = vs[:, :, 200:]
#world_size[2] = 50 # restrict to first 50 time steps
data_dict = {}
data_dict['v'] = vs # because scaling could be wrong
v_obs = Observable('v', 1)
observables = [v_obs]

# fix random seed
np.random.seed(1)

# initial setup of dataset
srd = SRDataset(world_size, data_dict, positions, observables, 
                kernel_sigma=0.01, cg_res=cg_res, deltat=deltat, cutoff=6)
#kernel_sigma=0.01/0.04 (lamp),0.1 (Jason?),0.6 (burgers?)

# initialize libraries, domains, and weights
srd.make_libraries(max_complexity=4, max_observables=3)
#srd.make_libraries(max_complexity=5, max_observables=4)
#srd.make_libraries(max_complexity=6, max_observables=4)

dom_width = 0.2#0.2 - LJ (lamp); 1.5 - Jason; 10 - Burgers?; 80 - ?
dom_time = 20 #20 - LJ; 30/50 - Burgers
srd.make_domains(ndomains=30, domain_size=[dom_width, dom_width, dom_time], pad=10)
#srd.make_domains(ndomains=1, domain_size=[dom_width, dom_width, dom_time], pad=0)
srd.make_weights(m=16, qmax=0) #qmax=1
srd.make_library_matrices(debug=False)

save('Q_bu.npy', srd.dxs, srd.libs)

[   1.01892117    1.01892117 1001.        ]
 

         944551172 function calls (938377299 primitive calls) in 3717.574 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  6042520  590.630    0.000 2064.639    0.000 _distn_infrastructure.py:1926(cdf)
  6042520  441.745    0.000 2926.073    0.000 convolution.py:16(gauss1d)
      300  378.874    1.263 3710.056   12.367 process_library_terms.py:187(eval_cgp)
 24184892  259.590    0.000  259.590    0.000 {method 'reduce' of 'numpy.ufunc' objects}
 12085040  197.176    0.000  275.632    0.000 numerictypes.py:575(_can_coerce_all)
60462923/60442734  172.340    0.000 1140.384    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
 24175194  153.587    0.000  549.229    0.000 fromnumeric.py:69(_wrapreduction)
  6042520  112.247    0.000  305.708    0.000 _distn_infrastructure.py:997(_open_support_mask)
  6042520  100.757    0.000  100.757    0.000 {method 'outer' of 'numpy.ufunc' objects}
  6056322   91.

In [3]:
_, libs = load('Q_bu.npy', 2)
libs = libs.item()

In [14]:
from identify_models import *
opts = dict()
opts['threshold'] = "'verbose'"
opts['row_norms'] = libs[0].row_weights
opts['gamma'] = 2
opts['verbose'] = False
opts['delta'] = 1e-8#1e-8 # apparently this reaches machine precision as the relative error
opts['char_sizes'] = libs[0].col_weights
opts['anchor_norm'] = np.linalg.norm(libs[0].Q[:, 0])
#opts['verbose'] = True
opts1 = opts.copy()
opts1['char_sizes'] = libs[1].col_weights
opts1['row_norms'] = libs[1].row_weights

#Q = libs[0].Q
#print(Q[:, 3]+Q[:, 14], np.linalg.norm(Q[:, 3]+Q[:, 14]), 
#np.linalg.norm(Q[:, 3]+Q[:, 14])/np.linalg.norm(Q[:, 0]))
#print(np.linalg.norm(Q[:, 0])/libs[0].col_weights[0])
#print(list(zip(libs[1].terms, libs[1].col_weights)))
#eqs0, lambdas0, derived_eqs0, excluded_terms = identify_equations(Q0, opts, terms0, observables, threshold=1e-5, 
#                       max_complexity=None, max_equations=999, timed=True)
#print(excluded_terms)
#eqs1, lambdas1, derived_eqs1, excluded_terms = identify_equations(Q1, opts1, terms1, observables, threshold=1e-5, 
#                       max_complexity=None, max_equations=999, timed=True, excluded_terms=excluded_terms)
#print(excluded_terms)
eqs, lambdas, derived_eqs, excluded_terms = interleave_identify([libs[0].Q, libs[1].Q], 
[opts, opts1], [libs[0].terms, libs[1].terms], threshold=5e-3)


#[0.15 s]
#Identified model: -0.9787526520510974 * rho + 0.028230207633854636 * rho * rho + -0.028610932601784253 * rho * rho[v_j * v_j] + 1.0 * rho[v_j * v_j] = 0 (order 5, residual 5.07e-05)
#[0.15 s]
#Identified model: -0.02837528778491848 * rho * rho + 1.0 * rho[v_j * v_j] = 0 (order 6, residual 1.43e-05)
#[0.26 s]
#Identified model: 0.9999199152531711 * dj rho[v_j] + 1.0 * dt rho = 0 (order 6, residual 3.51e-06)

[0.00 s]
Identified model: -6.552678376928359e-07 * rho * rho + 1.0 * rho[v_j * v_j] = 0 (order 3, residual 5.48e-04)
[0.01 s]
Identified model: 0.998766024887812 * dj rho[v_j] + 1.0 * dt rho = 0 (order 3, residual 2.56e-03)
[0.00 s]
Identified model: -0.0006002167565432877 * rho * rho + 1.0 * rho[v_j] * rho[v_j] = 0 (order 4, residual 1.62e-04)
[0.01 s]
Identified model: 1.0 * rho[v_i * v_j * v_j] = 0 (order 4, residual 5.99e-05)
[0.01 s]
Identified model: -0.00014119297646305562 * rho * rho[v_i] + 1.0 * rho[v_i] = 0 (order 4, residual 1.96e-03)


In [41]:
print(np.std(vs))

1.306835700243801


In [6]:
## mostly not planning to use this code

from ipynb.fs.full.sparse_reg import *
opts = dict()
opts['threshold'] = "'pareto'"
#opts['verbose'] = True
#opts['brute_force'] = False
#opts['threshold'] = "'threshold'"
#opts['n_terms'] = len(terms0)
#opts['gamma'] = 1.2
opts['char_sizes'] = srd.libs[0].col_weights
Xi, lambd, best_term, lambda1 = sparse_reg(srd.libs[0].Q, opts=opts)
print(srd.libs[0].Q.shape)

(240, 28)


In [8]:
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {srd.libs[0].terms[i]}")
print("Model residual:", lambd)
print("One-term model residual:", lambda1)
print(f"Best term (#{best_term}):", srd.libs[0].terms[best_term])

[Term 19] 1.0 * rho[v_j] * rho[v_j]
Model residual: 0.0030944418121483697
One-term model residual: 1.294046607514735e-05
Best term (#6): dt^3 rho


In [ ]:
opts['verbose'] = True
opts['char_sizes'] = srd.libs[1].col_weights
opts['avoid'] = []
opts['row_norms'] = None
Xi, lambd, best_term, lambda1 = sparse_reg(srd.libs[1].Q, opts=opts)
print(srd.libs[1].Q.shape)

In [ ]:
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {srd.libs[1].terms[i]}")
print("Model residual:", lambd)
print("One-term model residual:", lambda1)
print("Best term:", srd.libs[1].terms[best_term])

In [5]:
for term, size in zip(libs[0].terms, libs[0].col_weights):
    print(term, size)
for term, size in zip(libs[1].terms, libs[1].col_weights):
    print(term, size)

1 1
rho 4931.610740330862
dj^2 rho 2276.8170862054862
dt rho 2276.8170862054862
dt^2 rho 2276.8170862054862
rho * rho 24320784.494146716
rho * dt rho 11228375.596099796
rho * rho * rho 119940642024.60625
dj rho[v_j] 2923.728913619608
dt dj rho[v_j] 2923.728913619608
rho[v_j * v_j] 8132.174936496035
dt rho[v_j * v_j] 3754.447738523255
rho[v_j] * rho[v_j] 40104721.259073295
di rho 2276.8170862054862
dt di rho 2276.8170862054862
rho * di rho 11228375.596099796
rho[v_i] 6332.8288512380705
di dj rho[v_j] 2923.728913619608
dj^2 rho[v_i] 2923.728913619608
dt rho[v_i] 2923.728913619608
dt^2 rho[v_i] 2923.728913619608
rho * rho[v_i] 31231046.779442824
di rho[v_j * v_j] 3754.447738523255
dj rho[v_i * v_j] 3754.447738523255
rho[v_i * v_j * v_j] 10442.769061230103


In [6]:
continuity_terms = ['dt rho', 'dj rho[v_j]']
#continuity_terms = ['rho * dt rho', 'rho * dj rho[v_j]']
#continuity_terms = ['dt rho * rho', 'rho * rho * dj v_j', 'dj rho * rho * v_j']
#continuity_terms = ['dt rho', 'dj rv_j']
col_numbers = [find_term(libs[0].terms, name) for name in continuity_terms]
Xi, lambd = regress(libs[0].Q, col_numbers)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {libs[0].terms[i]}. (Char. size: {libs[0].col_weights[i]})")
print("Model residual:", lambd)

[Term 3] 1.0 * dt rho. (Char. size: 2276.8170862054862)
[Term 8] 0.99922409617847 * dj rho[v_j]. (Char. size: 2923.728913619608)
Model residual: 30.74135631624689


In [7]:
from scipy.stats.stats import pearsonr 
print(np.vstack([libs[0].Q[1:10, 3],libs[0].Q[1:10, 8]]))
print(libs[0].Q[1:10, 3]+libs[0].Q[1:10, 8])
print(libs[0].Q[0, 0])
print(pearsonr(libs[0].Q[1:10, 3], libs[0].Q[1:10, 8]))
print(np.mean(np.abs(libs[0].Q[:, 3]+libs[0].Q[:, 8])))
print('density:', np.mean(libs[0].Q[:, 1]/libs[0].Q[:, 0]), '+-', 
      np.std(libs[0].Q[:, 1]/libs[0].Q[:, 0])/np.sqrt(30))
print('ratio:', -np.mean(libs[0].Q[:, 3]/libs[0].Q[:, 8]), '+-', 
      np.std(libs[0].Q[:, 3]/libs[0].Q[:, 8])/np.sqrt(30))

[[  476814.74361695   321011.56586652   -65310.5205504  -1172085.83318433
    157218.91561871   447404.4734828   -386434.31446919   -48788.32041537
      8504.69389047]
 [ -474496.29842426  -316766.27699209    67441.28179599  1173635.22658714
   -160727.25622463  -450241.61807854   379853.75418006    50599.0940872
     -8303.08927101]]
[ 2318.44519268  4245.28887443  2130.7612456   1549.39340281
 -3508.34060592 -2837.14459574 -6580.56028913  1810.77367182
   201.60461945]
293.374863508793
(-0.9999767130676596, 2.0064123775463283e-16)
2930.634872431391
density: 5038.654661459805 +- 221.31532040083164
ratio: 1.000211656022018 +- 0.004320507618216155


In [10]:
print(np.linalg.norm(libs[0].Q[:, :], axis=0)/libs[0].col_weights)

[1.60688031e+03 1.68860148e+03 3.11999641e+06 1.64196067e+03
 1.56870066e+04 2.19347036e+03 2.27926979e+03 3.27056497e+03
 1.27964907e+03 1.20846445e+04 4.53909095e+00 2.48688685e+01
 8.22325083e-01]


In [9]:
burger_terms = ['dt rho[v_i]', 'dj rho[v_i * v_j]']
#burger_terms = ['rho[v_i]', 'rho[v_i * v_j * v_j]']
#burger_terms = ['rho * dt rv_i', 'dj rv_i * rv_j']
col_numbers = [find_term(libs[1].terms, name) for name in burger_terms]
Xi, lambd = regress(libs[1].Q, col_numbers)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {libs[1].terms[i]}. (Char. size: {libs[1].col_weights[i]})")
print("Model residual:", lambd)

[Term 6] 0.229149593949794 * dt rho[v_i]. (Char. size: 2923.728913619608)
[Term 10] 1.0 * dj rho[v_i * v_j]. (Char. size: 3754.447738523255)
Model residual: 0.009518828763897036


In [14]:
### not fixed (obviously)
#veqconst_terms = ['1', 'v_j * v_j']
veqconst_terms = ['rho', 'rho * v_j * v_j']
col_numbers = [find_term(terms0, name) for name in veqconst_terms]
Xi, lambd = regress(Q0, col_numbers)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {terms0[i]}. (Char. size: {char_sizes0[i]})")
print("Model residual:", lambd)

[Term 16] -0.004286563115825593 * rho. (Char. size: 0.1464261231728444)
[Term 28] 1.0 * rho * v_j * v_j. (Char. size: 0.0005121399208783936)
Model residual: 0.0005697337514007149


In [ ]:
rho dt^2 v = 10^-4 dt rho * grad rho